In [1]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
import json
from typing import Dict, List
# import pandas as pd
# from PyPDF2 import PdfReader
# import os
from pathlib import Path
import shutil
source = Path('../classroom/watsonx_utils.py')
destination = Path('./watsonx_utils.py')
shutil.copy(source, destination)
from watsonx_utils import get_sections
import sys
sys.path.append("..")

In [2]:
TEST_CASES = [
    {
        "Level": "grade1",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 3
    },
    {
        "Level": "grade1",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 4
    },
    {
        "Level": "grade1",
        "Disability": "no",
        "Subject": "science",
        "NumOfSections": 3
    },
    {
        "Level": "grade1",
        "Disability": "yes",
        "Subject": "math",
        "NumOfSections": 5
    },
    {
        "Level": "grade3",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 4
    },
    {
        "Level": "grade3",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 3
    },
    {
        "Level": "grade4",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 5
    },
    {
        "Level": "grade4",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 4
    },
    {
        "Level": "grade5",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 3
    },
    {
        "Level": "grade5",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 5
    },
    {
        "Level": "grade6",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 4
    },
    {
        "Level": "grade6",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 3
    },
    {
        "Level": "grade7",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 5
    },
    {
        "Level": "grade7",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 4
    },
    {
        "Level": "grade8",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 3
    },
    {
        "Level": "grade8",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 5
    },
    {
        "Level": "grade9",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 4
    },
    {
        "Level": "grade9",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 3
    },
    {
        "Level": "grade10",
        "Disability": "no",
        "Subject": "arabic",
        "NumOfSections": 5
    },
    {
        "Level": "grade10",
        "Disability": "yes",
        "Subject": "arabic",
        "NumOfSections": 4
    }
]


In [3]:
class CurriculumValidator:
    def __init__(self):
        """Initialize the validator with langchain components"""
        self.llm = ChatOllama(model='llama3', format="json", temperature=0)
        
        # Prompt for content validation
        self.validate_prompt = PromptTemplate(
            template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are an expert in educational content validation, the content is in arabic.
            Evaluate if the following topic and description are appropriate for:
            Grade Level: {level}
            Subject: {subject}
            Disability Accommodation Needed: {disability}

            Topic: {topic}
            Description: {description}

            Return only a JSON object with this structure:
            {{"is_valid": true/false}}
            \n<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """,
            input_variables=["level", "subject", "disability", "topic", "description"]
        )

    def validate_topic(self, topic_data: Dict, params: Dict) -> Dict:
        """
        Validate a single topic's relevancy and appropriateness
        
        Args:
            topic_data (dict): Contains Topic and Description
            params (dict): Contains Level, Subject, and Disability
        
        Returns:
            dict: Validation result with is_valid and reason
        """
        try:
            prompt = self.validate_prompt.format(
                level=params['Level'].replace("grade", "grade "),
                subject=params['Subject'],
                disability=params['Disability'],
                topic=topic_data['Topic'],
                description=topic_data['Description']
            )
            response = self.llm.predict(prompt)
            return json.loads(response)
        except Exception as e:
            return {
                "is_valid": False,
                "reason": f"Validation error: {str(e)}"
            }

    
    def get_key(self, doc, the_key = "is_valid"):
        keys = [k for k in doc.keys()]
        small_keys = [k.lower() for k in keys]
        for sk, k in zip(small_keys, keys):
            if the_key == sk:
                return k
        return  the_key
    
    def validate_curriculum(self, sections_data: Dict, params: Dict) -> Dict:
        """
        Validate the entire curriculum sections
        
        Args:
            sections_data (dict): The data returned by get_sections
            params (dict): Original parameters used for generation
        
        Returns:
            dict: Validation results including overall and per-topic validation
        """
        sections_data = {"data" : sections_data}
        results = {
            "params": params,
            "num_sections_match": False,
            "sections_validation": [],
            "overall_valid": False
        }
        
        # Check if data key exists
        if "data" not in sections_data:
            results["error"] = "Invalid format: 'data' key missing"
            return results
        
        # Validate number of sections
        actual_sections = len(sections_data["data"])
        expected_sections = params["NumOfSections"]
        results["num_sections_match"] = actual_sections == expected_sections
        
        if not results["num_sections_match"]:
            results["error"] = f"Section count mismatch. Expected: {expected_sections}, Got: {actual_sections}"
            return results
        
        # Validate each topic
        valid_topics = 0
        for topic_data in sections_data["data"]:
            validation_result = self.validate_topic(topic_data, params)
            results["sections_validation"].append({
                "topic": topic_data["Topic"],
                "validation": validation_result
            })
            the_key = self.get_key(validation_result)
            if validation_result[the_key]:
                valid_topics += 1
        
        # Set overall validation result
        results["overall_valid"] = (valid_topics == expected_sections)
        results["valid_topics_count"] = valid_topics
        
        return results



In [4]:
# validator = CurriculumValidator()
# params = TEST_CASES[0]

In [5]:
# sections = get_sections(params, '..\\educational_resources')
# sections

In [6]:
test_cases = TEST_CASES
validator = CurriculumValidator()
validation_results = []

for i, params in enumerate(test_cases, 1):
    try:
        print(f"\nValidating test case {i}/{len(test_cases)}")
        print(f"Parameters: {params}")
        
        # Get sections using provided function
        sections = get_sections(params, '..\\educational_resources')
        
        # Validate the sections
        result = validator.validate_curriculum(sections, params)
        
        validation_results.append(result)
        
        # Print validation summary
        print(f"Sections count match: {result['num_sections_match']}")
        print(f"Valid topics: {result.get('valid_topics_count', 0)}/{params['NumOfSections']}")
        print(f"Overall valid: {result['overall_valid']}")
        
    except Exception as e:
        print(f"Error in test case {i}: {str(e)}")
        validation_results.append({
            "params": params,
            "error": str(e),
            "overall_valid": False
        })

# Print final summary
successful_cases = sum(1 for r in validation_results if r.get("overall_valid", False))
print(f"\nValidation Summary:")
print(f"Successful cases: {successful_cases}/{len(test_cases)}")
print(f"Failed cases: {len(test_cases) - successful_cases}/{len(test_cases)}")
validation_results

C:\Users\user\AppData\Local\Temp\ipykernel_7960\3416293957.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model='llama3', format="json", temperature=0)



Validating test case 1/20
Parameters: {'Level': 'grade1', 'Disability': 'no', 'Subject': 'arabic', 'NumOfSections': 3}
{'data': [{'Topic': 'القسم الأول: تعلم الحروف العربية (الأبجدية)', 'Description': 'تعلم الحروف العربية بالترتيب الأبجدي:\n- حرف الألف (ا)\n- حرف الباء (ب)\n- حرف التاء (ت)\n- ... وصولاً إلى حرف الياء (ي)'}, {'Topic': 'القسم الثاني: تعلم أصوات الحروف (الصوتيات)', 'Description': 'تعلم أصوات الحروف العربية:\n- حرف الألف (ا) - صوت طويل (ا)\n- حرف الباء (ب) - صوت قصير (ب)\n- ... وصولاً إلى حرف الياء (ي)'}, {'Topic': 'مراجعة الحروف', 'Description': 'مراجعة الحروف العربية مع الطلاب للتأكد من فهمهم واستيعابهم لها.\nاستخدم تمارين بسيطة مثل كتابة الحروف على السبورة أو على ورق العمل أو استخدام تطبيقات تعليمية تفاعلية لتعزيز التعلم.'}]}


C:\Users\user\AppData\Local\Temp\ipykernel_7960\3416293957.py:44: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm.predict(prompt)


Sections count match: True
Valid topics: 2/3
Overall valid: False

Validating test case 2/20
Parameters: {'Level': 'grade1', 'Disability': 'yes', 'Subject': 'arabic', 'NumOfSections': 4}
{'data': [{'Topic': 'القسم الأول: تعلم الحروف العربية الأساسية', 'Description': 'تعلم الحروف العربية الأساسية\n1. المقدمة: ابدأ بشرح أهمية تعلم الحروف العربية وكيف تساعدنا في التواصل والقراءة والكتابة باللغة العربية.\n2. تعريف الحروف: قدم تعريفًا موجزًا لكل حرف من الحروف المذكورة (م, ب, ل, د, ن, ر, ص, ف, س, ق, ت, ح) واطلب من الطلاب تكرار كل حرف بصوت عالٍ.\n3. كتابة الحروف: اطلب من الطلاب كتابة كل حرف على ورقة بشكل فردي، مع التأكيد على الشكل الصحيح لكل حرف.\n4. تمارين التذكر: قدم تمارين بسيطة تساعد الطلاب على تذكر الحروف، مثل ربط كل حرف بصورة أو كلمة تبدأ بنفس الحرف.\n5. مراجعة: قم بمراجعة الحروف مع الطلاب بشكل دوري للتأكد من فهمهم واستيعابهم لها.'}, {'Topic': 'القسم الثاني: تعلم الأصوات القصيرة والطويلة', 'Description': 'تعلم الأصوات القصيرة والطويلة\n1. المقدمة: اشرح للطلاب أن الحروف العربية لها أصوات

[{'params': {'Level': 'grade1',
   'Disability': 'no',
   'Subject': 'arabic',
   'NumOfSections': 3},
  'num_sections_match': True,
  'sections_validation': [{'topic': 'القسم الأول: تعلم الحروف العربية (الأبجدية)',
    'validation': {'Is_Valid': True}},
   {'topic': 'القسم الثاني: تعلم أصوات الحروف (الصوتيات)',
    'validation': {'Is_Valid': False}},
   {'topic': 'مراجعة الحروف', 'validation': {'Is_Valid': True}}],
  'overall_valid': False,
  'valid_topics_count': 2},
 {'params': {'Level': 'grade1',
   'Disability': 'yes',
   'Subject': 'arabic',
   'NumOfSections': 4},
  'num_sections_match': True,
  'sections_validation': [{'topic': 'القسم الأول: تعلم الحروف العربية الأساسية',
    'validation': {'Is_Valid': True}},
   {'topic': 'القسم الثاني: تعلم الأصوات القصيرة والطويلة',
    'validation': {'is_valid': False}},
   {'topic': 'القسم الثالث: تعلم الكلمات والجمل البسيطة',
    'validation': {'is_valid': True}},
   {'topic': 'القسم الرابع: تعلم القراءة والكتابة بشكل تفاعلي',
    'valida